In [2]:
import pandas as pd
from IPython.display import display

# Load data from CSV files
player_plays = pd.read_csv('../../data/player_play.csv')  # Player actions on each play
players = pd.read_csv('../../data/players.csv')  # Player metadata (name, position, etc.)
plays = pd.read_csv('../../data/plays.csv')  # Play-level information
games_data = pd.read_csv('../../data/games.csv')  # Game-level information

# Merge datasets to combine all necessary information

# Step 1: Combine plays with game data using 'gameId' as the key
plays_with_games = plays.merge(games_data, on='gameId', how='left')

# Step 2: Merge player_plays data with the previous merged plays_with_games dataset to include player-specific actions
combined_data = player_plays.merge(plays_with_games, on=['gameId', 'playId'], how='left')

# Step 3: Add player metadata (e.g., name, position) by merging with the players dataset using 'nflId' as the key
combined_data = combined_data.merge(players[['nflId', 'displayName', 'position']], on='nflId', how='left')

# Create a dictionary for fast lookup of player names from 'nflId'
nflId_to_displayName = players.set_index('nflId')['displayName'].to_dict()

# Define function to determine the "key player" for each play
# A key player is the player who performed a significant action like rushing, receiving, or dropping back
def determine_key_player(group):
    if not group[group['hadRushAttempt'] == 1].empty:
        return group.loc[group['hadRushAttempt'] == 1, 'nflId'].iloc[0]  # First player with rush attempt
    elif not group[(group['hadPassReception'] == 1) | (group['wasTargettedReceiver'] == 1)].empty:
        return group.loc[(group['hadPassReception'] == 1) | (group['wasTargettedReceiver'] == 1), 'nflId'].iloc[0]  # First player with pass reception or target
    elif not group[group['hadDropback'] == 1].empty:
        return group.loc[group['hadDropback'] == 1, 'nflId'].iloc[0]  # First player with dropback
    return None  # Return None if no key player found

# Define a function to aggregate data for each play
def aggregate_play(play_df):
    # Extract static play-level fields such as gameId, playId, week, and play description
    play_info = {
        'gameId': play_df['gameId'].iloc[0],
        'playId': play_df['playId'].iloc[0],
        'week': play_df['week'].iloc[0],
        'quarter': play_df['quarter'].iloc[0],
        'gameClock': play_df['gameClock'].iloc[0],
        'down': play_df['down'].iloc[0],
        'yardsToGo': play_df['yardsToGo'].iloc[0],
        'yardsGained': play_df['yardsGained'].iloc[0],
        'playDescription': play_df['playDescription'].iloc[0],
        'possessionTeam': play_df['possessionTeam'].iloc[0],
        'homeTeamAbbr': play_df['homeTeamAbbr'].iloc[0],
        'visitorTeamAbbr': play_df['visitorTeamAbbr'].iloc[0],
        'preSnapHomeScore': play_df['preSnapHomeScore'].iloc[0],
        'preSnapVisitorScore': play_df['preSnapVisitorScore'].iloc[0],
    }
    
    # Use the determine_key_player function to find the key player for this play and map to player name
    key_player_id = determine_key_player(play_df)
    play_info['keyPlayer'] = nflId_to_displayName[key_player_id] if key_player_id else None

    # Determine players in motion at the ball snap (if any)
    motion_player_rows = play_df.loc[play_df['inMotionAtBallSnap'] == True]
    play_info['motion_players'] = [
        nflId_to_displayName[nflId] for nflId in motion_player_rows['nflId'].tolist()
    ] if not motion_player_rows.empty else None

    # Determine players involved in pre-snap motion or shifts, but not in motion at the snap
    shift_player_rows = play_df.loc[
        ((play_df['motionSinceLineset'] == True) | (play_df['shiftSinceLineset'] == True)) & 
        (play_df['inMotionAtBallSnap'] == False)
    ]
    play_info['shift_players'] = [
        nflId_to_displayName[nflId] for nflId in shift_player_rows['nflId'].tolist()
    ] if not shift_player_rows.empty else None
    
    # Aggregate boolean columns to indicate if the condition applies to any player
    play_info['inMotionAtBallSnap'] = play_df['inMotionAtBallSnap'].any()
    play_info['motionSinceLineset'] = play_df['motionSinceLineset'].any()
    play_info['shiftSinceLineset'] = play_df['shiftSinceLineset'].any()
    
    # Add more columns from the 'plays' dataframe for additional play details
    play_info['absoluteYardlineNumber'] = play_df['absoluteYardlineNumber'].iloc[0]
    play_info['yardlineSide'] = play_df['yardlineSide'].iloc[0]
    play_info['passResult'] = play_df['passResult'].iloc[0]
    play_info['isDropback'] = play_df['isDropback'].iloc[0]
    play_info['yardlineNumber'] = play_df['yardlineNumber'].iloc[0]
    play_info['expectedPoints'] = play_df['expectedPoints'].iloc[0]
    play_info['expectedPointsAdded'] = play_df['expectedPointsAdded'].iloc[0]
    play_info['playNullifiedByPenalty'] = play_df['playNullifiedByPenalty'].iloc[0]
    play_info['preSnapHomeTeamWinProbability'] = play_df['preSnapHomeTeamWinProbability'].iloc[0]
    play_info['preSnapVisitorTeamWinProbability'] = play_df['preSnapVisitorTeamWinProbability'].iloc[0]
    play_info['homeTeamWinProbabilityAdded'] = play_df['homeTeamWinProbabilityAdded'].iloc[0]
    play_info['visitorTeamWinProbabilityAdded'] = play_df['visitorTeamWinProbabilityAdded'].iloc[0]
    play_info['pff_passCoverage'] = play_df['pff_passCoverage'].iloc[0]
    play_info['pff_manZone'] = play_df['pff_manZone'].iloc[0]
    play_info['offenseFormation'] = play_df['offenseFormation'].iloc[0]

    # Add game-level data, like final scores
    play_info['homeFinalScore'] = play_df['homeFinalScore'].iloc[0]
    play_info['visitorFinalScore'] = play_df['visitorFinalScore'].iloc[0]

    # Add player-level data, like whether a fumble was lost on this play
    fumble_lost_row = play_df.loc[play_df['fumbleLost'] == 1]
    play_info['fumbleLost'] = fumble_lost_row['fumbleLost'].iloc[0] if not fumble_lost_row.empty else None

    return play_info

# Group data by 'gameId' and 'playId', then apply aggregation function
aggregated_plays = combined_data.groupby(['gameId', 'playId']).apply(aggregate_play).apply(pd.Series)

# Optional: Save to CSV or inspect the final aggregated data
#aggregated_plays.to_csv("complete_merge_preMetrics.csv", index=False)

# Output the column names of the aggregated data for inspection
print(list(aggregated_plays.columns))  # To check the columns in the final dataframe
display(aggregated_plays.head())  # Display the first few rows of the aggregated dataframe


['gameId', 'playId', 'week', 'quarter', 'gameClock', 'down', 'yardsToGo', 'yardsGained', 'playDescription', 'possessionTeam', 'homeTeamAbbr', 'visitorTeamAbbr', 'preSnapHomeScore', 'preSnapVisitorScore', 'keyPlayer', 'motion_players', 'shift_players', 'inMotionAtBallSnap', 'motionSinceLineset', 'shiftSinceLineset', 'absoluteYardlineNumber', 'yardlineSide', 'passResult', 'isDropback', 'yardlineNumber', 'expectedPoints', 'expectedPointsAdded', 'playNullifiedByPenalty', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability', 'homeTeamWinProbabilityAdded', 'visitorTeamWinProbabilityAdded', 'pff_passCoverage', 'pff_manZone', 'offenseFormation', 'homeFinalScore', 'visitorFinalScore', 'fumbleLost']


gameId  playId  week  quarter gameClock  down  \
gameId     playId                                                      
2022090800 56      2022090800      56     1        1     15:00     1   
           80      2022090800      80     1        1     14:29     2   
           101     2022090800     101     1        1     13:54     1   
           122     2022090800     122     1        1     13:15     2   
           167     2022090800     167     1        1     11:54     2   

                   yardsToGo  yardsGained  \
gameId     playId                           
2022090800 56             10            6   
           80              4            7   
           101            10            7   
           122             3            6   
           167             8           12   

                                                     playDescription  \
gameId     playId                                                      
2022090800 56      (15:00) (Shotgun) J.Allen pass short right to ...   
           80      (14:29) (No Huddle, Shotgun) J.Allen scrambles...   
           101     (13:54) D.Singletary right end to BUF 45 for 7...   
           122     (13:15) (Shotgun) J.Allen pass short right to ...   
           167     (11:54) (Shotgun) J.Allen pass short middle to...   

                  possessionTeam  ... preSnapHomeTeamWinProbability  \
gameId     playId                 ...                                 
2022090800 56                BUF  ...                      0.413347   
           80                BUF  ...                      0.413316   
           101               BUF  ...                      0.399819   
           122               BUF  ...                      0.384969   
           167               BUF  ...                      0.387554   

                  preSnapVisitorTeamWinProbability  \
gameId     playId                                    
2022090800 56                             0.586653   
           80                             0.586684   
           101                            0.600181   
           122                            0.615031   
           167                            0.612446   

                   homeTeamWinProbabilityAdded  \
gameId     playId                                
2022090800 56                        -0.000031   
           80                        -0.013497   
           101                       -0.014850   
           122                       -0.019032   
           167                       -0.031870   

                   visitorTeamWinProbabilityAdded pff_passCoverage  \
gameId     playId                                                    
2022090800 56                            0.000031     Cover 6-Left   
           80                            0.013497     Cover 6-Left   
           101                           0.014850    Cover-6 Right   
           122                           0.019032          Cover-3   
           167                           0.031870          Cover-3   

                  pff_manZone offenseFormation  homeFinalScore  \
gameId     playId                                                
2022090800 56            Zone          SHOTGUN              10   
           80            Zone            EMPTY              10   
           101           Zone           I_FORM              10   
           122           Zone          SHOTGUN              10   
           167           Zone            EMPTY              10   

                   visitorFinalScore  fumbleLost  
gameId     playId                                 
2022090800 56                     31         NaN  
           80                     31         NaN  
           101                    31         NaN  
           122                    31         NaN  
           167                    31         NaN  

[5 rows x 38 columns]